In [6]:
# Train/Test RNN
import h5py
import numpy as np
import pickle
from inputs import Data, load_separators, _build_bin_output, build_IO_from_var, build_DTA_from_var
from RNN import modelRNN
from features import get_init_end_dates, get_group_name

dateTest = ([                                                       '2018.03.09',
                '2018.03.12','2018.03.13','2018.03.14','2018.03.15','2018.03.16',
                '2018.03.19','2018.03.20','2018.03.21','2018.03.22','2018.03.23',
                '2018.03.26','2018.03.27','2018.03.28','2018.03.29','2018.03.30',
                '2018.04.02','2018.04.03','2018.04.04','2018.04.05','2018.04.06',
                '2018.04.09','2018.04.10','2018.04.11','2018.04.12','2018.04.13',
                '2018.04.16','2018.04.17','2018.04.18','2018.04.19','2018.04.20',
                '2018.04.23','2018.04.24','2018.04.25','2018.04.26','2018.04.27',
                '2018.04.30','2018.05.01','2018.05.02','2018.05.03','2018.05.04',
                '2018.05.07','2018.05.08','2018.05.09','2018.05.10','2018.05.11',
                '2018.05.14','2018.05.15','2018.05.16','2018.05.17','2018.05.18',
                '2018.05.21','2018.05.22','2018.05.23','2018.05.24','2018.05.25',
                '2018.05.28','2018.05.29','2018.05.30','2018.05.31','2018.06.01',
                '2018.06.04','2018.06.05','2018.06.06','2018.06.07','2018.06.08',
                '2018.06.11','2018.06.12','2018.06.13','2018.06.14','2018.06.15',
                '2018.06.18','2018.06.19','2018.06.20','2018.06.21','2018.06.22',
                '2018.06.25','2018.06.26','2018.06.27','2018.06.28','2018.06.29',
                '2018.07.02','2018.07.03','2018.07.04','2018.07.05','2018.07.06',
                '2018.07.09','2018.07.10','2018.07.11','2018.07.12','2018.07.13',
                '2018.07.30','2018.07.31','2018.08.01','2018.08.02','2018.08.03',
                '2018.08.06','2018.08.07','2018.08.08','2018.08.09','2018.08.10']+
               ['2018.08.13','2018.08.14','2018.08.15','2018.08.16','2018.08.17',
                '2018.08.20','2018.08.21','2018.08.22','2018.08.23','2018.08.24',
                '2018.08.27','2018.08.28','2018.08.29','2018.08.30','2018.08.31',
                '2018.09.03','2018.09.04','2018.09.05','2018.09.06','2018.09.07',
                '2018.09.10','2018.09.11','2018.09.12','2018.09.13','2018.09.14',
                '2018.09.17','2018.09.18','2018.09.19','2018.09.20','2018.09.21',
                '2018.09.24','2018.09.25','2018.09.26','2018.09.27']+['2018.09.28',
                '2018.10.01','2018.10.02','2018.10.03','2018.10.04','2018.10.05',
                '2018.10.08','2018.10.09','2018.10.10','2018.10.11','2018.10.12',
                '2018.10.15','2018.10.16','2018.10.17','2018.10.18','2018.10.19',
                '2018.10.22','2018.10.23','2018.10.24','2018.10.25','2018.10.26',
                '2018.10.29','2018.10.30','2018.10.31','2018.11.01','2018.11.02',
                '2018.11.05','2018.11.06','2018.11.07','2018.11.08','2018.11.09'])

data=Data(movingWindow=100,
          nEventsPerStat=1000,
          lB=1300, 
          dateTest=dateTest,
          assets=[1,2,3,4,7,8,10,11,12,13,14,15,16,17,19,27,28,29,30,31,32],
          channels=[0],
          max_var=10,
          feature_keys_manual=[],
          feature_keys_tsfresh=[],#[i for i in range(37,48)]+[49,50]+[i for i in range(52,68)],
          var_feat_keys=[i for i in range(68,93)])

model=modelRNN(data,
               size_hidden_layer=100,
               L=3,
               size_output_layer=5,
               keep_prob_dropout=1,
               miniBatchSize=32,
               outputGain=.6,
               commonY=3,
               lR0=0.0001,
               num_epochs=20)

tOt = 'tr' # tr->train te->test
if tOt == 'tr':
    calculate_roi = False
else:
    calculate_roi = True
if_build_IO = False
from_stats_file = False

IDweights = '000400'
IDresults = '100400'
IO_results_name = IDresults
hdf5_directory = 'D:/SDC/py/HDF5/'
feats_var_directory = hdf5_directory+'feats_var/'
IO_directory = '../RNN/IO/'
if tOt=='tr':
    filename_IO = IO_directory+'IO_'+IDweights+'.hdf5'
else:
    filename_IO = IO_directory+'IO_'+IO_results_name+'.hdf5'
separators_directory = hdf5_directory+'separators/'

# if IO structures have to be built 
if if_build_IO:
    # init dictionary containing IO structures
    IO = {}
    # open IO file for writting
    f_IO = h5py.File(filename_IO,'w')
    # init IO data sets
    X = f_IO.create_dataset('X',
                            (0, model.seq_len, model.nFeatures), 
                            maxshape=(None,model.seq_len, model.nFeatures), 
                            dtype=float)
    Y = f_IO.create_dataset('Y',
                            (0,model.seq_len,model.commonY+model.size_output_layer),
                            maxshape=(None,model.seq_len,model.commonY+
                            model.size_output_layer),
                            dtype=float)
    if calculate_roi:
        D = f_IO.create_dataset('D', (0,2),
                                    maxshape=(None,2),dtype='S19')
        B = f_IO.create_dataset('B', (0,2),
                                    maxshape=(None,2),dtype=float)
        A = f_IO.create_dataset('A', (0,2),
                                    maxshape=(None,2),dtype=float)
        IO['D'] = D
        IO['B'] = B
        IO['A'] = A
    # attributes to track asset-IO belonging
    ass_IO_ass = np.zeros((len(data.assets))).astype(int)
    # structure that tracks the number of samples per level
    totalSampsPerLevel = np.zeros((model.size_output_layer))
    # save IO structures in dictionary
    IO['X'] = X
    IO['Y'] = Y
    IO['pointer'] = 0
    
for ass_idx, ass in enumerate(data.assets):
    thisAsset = data.AllAssets[str(ass)]
    print(str(ass)+" "+thisAsset)
    #thisAsset = 'EURUSD'
    # load separators
    separators = load_separators(data, 
                                 thisAsset, 
                                 separators_directory, 
                                 tOt='tr', 
                                 from_txt=1)
    aloc = 2**20
    # index asset
    #ass_idx = 0

    filename_features = (feats_var_directory+thisAsset+'_feats_var_mW'+str(data.movingWindow)+'_nE'+
                                str(data.nEventsPerStat)+'.hdf5')
    file_features = h5py.File(filename_features,'r')
    filename_returns = (feats_var_directory+thisAsset+'_rets_var_mW'+str(data.movingWindow)+'_nE'+
                                str(data.nEventsPerStat)+'.hdf5')
    file_returns = h5py.File(filename_returns,'r')
    filename_symbols = (feats_var_directory+thisAsset+'_symbols_mW'+str(data.movingWindow)+'_nE'+
                                str(data.nEventsPerStat)+'.hdf5')
    file_symbols = h5py.File(filename_symbols,'r')
    filename_stats = (feats_var_directory+thisAsset+'_stats_mW'+str(data.movingWindow)+'_nE'+
                                    str(data.nEventsPerStat)+'.p')
    # init or load total stats
    if not from_stats_file:
        stats = {}
        # load stats in
        stats["means_in"] = file_features[thisAsset].attrs.get("means_in")
        stats["stds_in"] = file_features[thisAsset].attrs.get("stds_in")
        stats["m_in"] = file_features[thisAsset].attrs.get("m_in")
        # load stats out
        stats["means_out"] = file_returns[thisAsset].attrs.get("means_out")
        stats["stds_out"] = file_returns[thisAsset].attrs.get("stds_out")
        stats["m_out"] = file_returns[thisAsset].attrs.get("m_out")
    elif from_stats_file:
        stats = pickle.load( open( filename_stats, "rb" ))
    print(stats)

    nExS = data.nEventsPerStat
    mW = data.movingWindow
    if if_build_IO:
        for s in range(0,len(separators)-1,2):#
            print("\ts {0:d} of {1:d}".format(int(s/2),int(len(separators)/2-1))+
                  ". From "+separators.DateTime.iloc[s]+" to "+
                  separators.DateTime.iloc[s+1])

            # number of events within this separator chunk
            nE = separators.index[s+1]-separators.index[s]+1
            # get first day after separator
            day_s = separators.DateTime.iloc[s][0:10]
            # check if number of events is not enough to build two features and one return
            if nE-nExS>=2*nExS:
                if (tOt == 'tr' and (day_s not in data.dateTest and day_s<=data.dateTest[-1])) or \
                    (tOt == 'te' and (day_s in data.dateTest and day_s<=data.dateTest[-1])):
                    # init and end dates
                    init_date, end_date = get_init_end_dates(separators, s)
                    #print("init_date")
                    #print(init_date)
                    #print("end_date")
                    #print(end_date)
                    # get group name
                    group_name = get_group_name(thisAsset, init_date, end_date)
                    #print("group_name")
                    #print(group_name)
                    # load features
                    if group_name in file_features:
                        features = file_features[group_name]["features"]
                    else:
                        raise ValueError(group_name+" not in "+filename_features)
                    #print("features")
                    #print(features)
                    # load returns
                    if group_name in file_returns:
                        returns = file_returns[group_name]["returns"]
                    else:
                        raise ValueError(group_name+" not in "+filename_features)
                    #print(returns)
                    # load Symbols if calculate_roi is true
                    #TODO: Implement Symbol loading for test
                    if calculate_roi:
                        DT = file_symbols[group_name]["DT"]
                        B = file_symbols[group_name]["B"]
                        A = file_symbols[group_name]["A"]
                        # build dict w. symbols
                        symbols = {'DT':DT,
                                  'B':B,
                                  'A':A}
                    else:
                        symbols = {}
                    # build IO
                    IO, totalSampsPerLevel = build_IO_from_var_notebook(data, model, stats, IO, totalSampsPerLevel, 
                                                                        features, returns, symbols, calculate_roi)
                    print(totalSampsPerLevel)
                else:
                    print("\tNot in the set. Skipped.")

            else:
                print("\ts {0:d} of {1:d}. Not enough entries. Skipped.".format(
                    int(s/2),int(len(separators)/2-1)))
        # end of for s in range(0,len(separators)-1,2):
    if if_build_IO:
        ass_IO_ass[ass_idx] = IO['pointer']
        #print("\tTime for "+thisAsset+":"+str(np.floor(time.time()-tic))+"s"+
        #          ". Total time:"+str(np.floor(time.time()-ticTotal))+"s")
# end of for ass_idx, ass in enumerate(data.assets):
if if_build_IO:
    print("Samples to RNN: "+str(max(ass_IO_ass))+".\nPercent per level:"+
            str(totalSampsPerLevel/max(ass_IO_ass)))
    f_IO.attrs.create('ass_IO_ass', ass_IO_ass, dtype=int)
    
    
    if calculate_roi:
        print("Building DTA...")
        DTA = build_DTA_from_var(data, IO['D'], IO['B'], IO['A'], ass_IO_ass)
        pickle.dump( DTA, open( "../RNN/IO/DTA"+"_"+IO_results_name+".p", "wb" ))
        f_IO.attrs.create('ass_IO_ass', ass_IO_ass, dtype=int)
        
    f_IO.close()
else:
    if calculate_roi:
        DTA = pickle.load( open( "../RNN/IO/DTA"+"_"+IO_results_name+".p", "rb" ))
    # get ass_IO_ass from disk
    f_IO = h5py.File(filename_IO,'r')
    ass_IO_ass = f_IO.attrs.get("ass_IO_ass")
    f_IO.close()
m_t = max(ass_IO_ass)
print("DONE")

1 AUDCAD
{'means_in': array([[-4.36942530e-07, -4.47432327e-07, -4.37541144e-07,
        -4.21726903e-07, -2.50684735e-07, -1.27332027e-08,
         1.93838145e-06,  4.45454320e-06, -6.33446959e+01,
         7.77982540e+00,  5.31584223e-04, -5.31164977e-04,
         5.96836233e-04, -5.85379787e-04,  5.58178248e-01,
         5.00040691e-04, -5.01337145e-04,  1.29525496e+01,
         2.61798703e-03,  2.31730762e-02,  2.48865723e-02,
         6.42509688e-02,  9.67864073e-02,  2.16410337e-01,
         3.83013997e-01]]), 'means_out': array([-2.95589297e-08, -7.53936874e-08, -2.17893512e-07, -4.45835679e-07]), 'm_out': 868773, 'stds_out': array([0.00016564, 0.00022689, 0.00033817, 0.0004634 ]), 'stds_in': array([[4.63388284e-04, 4.34756482e-04, 3.39125208e-04, 2.69818604e-04,
        1.35063097e-04, 1.01298943e-04, 8.97005624e-05, 1.11987161e-04,
        3.96523083e+00, 2.27026236e+00, 4.30828392e-04, 4.35330111e-04,
        3.37656915e-04, 3.05043884e-04, 2.53610161e-01, 4.41937573e-04,
   

{'means_in': array([[-2.44071782e-06, -2.45113200e-06, -2.45132742e-06,
        -2.43273893e-06, -2.02737844e-06, -1.44250602e-06,
         3.29429007e-06,  8.88346746e-06, -5.79218237e+01,
         6.80122002e+00,  1.00674778e-03, -1.00773940e-03,
         1.13954810e-03, -1.11793225e-03,  5.68105987e-01,
         9.44094681e-04, -9.48764074e-04,  1.18345312e+01,
         1.08109785e-02,  7.10853596e-03,  5.12068678e-03,
         2.40689989e-02,  1.00224940e-02, -9.96860563e-03,
         1.29413890e-01]]), 'means_out': array([-2.38846261e-07, -4.91711620e-07, -1.24105619e-06, -2.48028221e-06]), 'm_out': 1092795, 'stds_out': array([0.00034527, 0.0004801 , 0.00071934, 0.0009845 ]), 'stds_in': array([[9.84504265e-04, 9.26328921e-04, 7.22101472e-04, 5.73528798e-04,
        2.82930868e-04, 2.08281219e-04, 1.51904734e-04, 1.78318095e-04,
        4.35780470e+00, 2.32675818e+00, 9.50925519e-04, 1.05129890e-03,
        9.00210306e-04, 7.56219965e-04, 2.43949374e-01, 9.62147569e-04,
        1.0

{'means_in': array([[-7.97532330e-06, -7.75451159e-06, -6.97335479e-06,
        -5.16685487e-06,  1.00725308e-05,  2.77287741e-05,
         1.77197947e-04,  3.81930506e-04, -2.32977113e+01,
         6.67583052e+00,  5.45766472e-02, -5.45323193e-02,
         6.04825194e-02, -5.98413265e-02,  5.20680843e-01,
         5.12982778e-02, -5.13950406e-02,  5.04970511e+00,
        -1.41365798e-03,  2.28339940e-02,  4.07388681e-02,
         1.30404146e-01,  1.88836249e-01,  2.31061604e-01,
         2.42853150e-01]]), 'means_out': array([-3.38992577e-07, -7.43851588e-07, -2.13143779e-06, -5.09630551e-06]), 'm_out': 1138668, 'stds_out': array([0.01715338, 0.02431066, 0.03718885, 0.05164517]), 'stds_in': array([[5.16475653e-02, 4.88073984e-02, 3.83307928e-02, 3.06279833e-02,
        1.52779820e-02, 1.11120447e-02, 6.30938001e-03, 5.82836014e-03,
        4.36978032e+00, 2.13593273e+00, 4.77018860e-02, 4.93426386e-02,
        3.64261588e-02, 3.57272636e-02, 2.63114466e-01, 4.86922310e-02,
        5.0

In [8]:
m_t

733966

In [8]:
# Train
import tensorflow as tf
# reset graph
alloc = 2**20
tf.reset_default_graph()
# start session
with tf.Session() as sess:    
    model.train(sess, int(np.ceil(m_t/aloc)), ID=IDweights, IDIO=IDweights, 
                data_format='hdf5', filename_IO=filename_IO, aloc=alloc)

ID = 000400
INFO:tensorflow:Restoring parameters from ../RNN/weights/000400/000005/000005
Parameters loaded. Epoch 5
Training from epoch 6 till 25
Time training Chunck 0 of 14: 383 s
Time training Chunck 1 of 14: 389 s
Time training Chunck 2 of 14: 402 s
Time training Chunck 3 of 14: 401 s
Time training Chunck 4 of 14: 404 s
Time training Chunck 5 of 14: 419 s
Time training Chunck 6 of 14: 402 s
Time training Chunck 7 of 14: 401 s
Time training Chunck 8 of 14: 417 s
Time training Chunck 9 of 14: 420 s
Time training Chunck 10 of 14: 414 s
Time training Chunck 11 of 14: 408 s
Time training Chunck 12 of 14: 391 s
Time training Chunck 13 of 14: 405 s
Time training Chunck 14 of 14: 359 s
Cost after epoch 6: 0.505045. Av cost 4.277871
13:37:19 Total time training: 100.38m
INFO:tensorflow:../RNN/weights/000400/000006/000006 is not in all_model_checkpoint_paths. Manually adding it.
Parameters saved
Time training Chunck 0 of 14: 398 s
Time training Chunck 1 of 14: 403 s
Time training Chunck 2 o

KeyboardInterrupt: 

In [5]:
# Test
import tensorflow as tf

alloc = 200000
save_journal = False
startFrom = -1
endAt = -1
# reset graph
tf.reset_default_graph()
# start session
with tf.Session() as sess:
    # run test RNN
    print("IDresults: "+IDresults)
    # TEMP: GRE calculation not implemented in test2 yet. Use old test
    if save_journal:
        model.test(sess, data, IDresults, IDweights, 
                    alloc, 'test', startFrom=startFrom,
                    IDIO=IO_results_name, data_format='hdf5', DTA=DTA, 
                    save_journal=save_journal, endAt=endAt)
    else:
        model.test2(sess, data, IDresults, IDweights, 
                    alloc, 'test', startFrom=startFrom,
                    IDIO=IO_results_name, data_format='hdf5', DTA=DTA, 
                    save_journal=save_journal, endAt=endAt, from_var=True)
print("DONE")

IDresults: 100400
INFO:tensorflow:Restoring parameters from ../RNN/weights/000400/000000/000000
Parameters loaded. Epoch 0
Epoch 0 of 5. Getting output...
Chunck 1 of 18
Chunck 2 of 18
Chunck 3 of 18
Chunck 4 of 18
Chunck 5 of 18
Chunck 6 of 18
Chunck 7 of 18
Chunck 8 of 18
Chunck 9 of 18
Chunck 10 of 18
Chunck 11 of 18
Chunck 12 of 18
Chunck 13 of 18
Chunck 14 of 18
Chunck 15 of 18
Chunck 16 of 18
Chunck 17 of 18
Chunck 18 of 18
Getting results
Epoch = 0. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.5
J_test = 4.2419524, J_train = 4.306788505555485, Accuracy=0.7324523401267535
RD = 12549 NZ = 24034 NZA = 41847 pNZ = 0.695% pNZA = 1.211% AD = 52.21% ADA = 29.99% NO = 4115 GSP = 57.76% NSP = 48.41%
SI2 = -262.25 SI = -271.25 eGROI = 30.72% eROI = -55.99% eROI2 = -51.58% eROI3 = -92.73% mSpread = 0.0211%
Epoch = 0. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.6
RD = 869 NZ = 1467 NZA = 2183 pNZ = 0.042% pNZA = 0.063% AD = 59.24% ADA = 39.81% NO = 801 GSP = 61.42% NSP = 51

C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\fromnumeric.py:3194: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch = 0. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.7
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 0. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.8
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 0. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 0. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.5
RD = 3410 NZ = 6563 NZA = 10118 pNZ = 0.190% pNZA = 0.293% AD = 51.96% ADA = 33.70% NO = 1420 GSP = 56.83% NSP = 47.82%
SI2 = -104.00 SI =

Epoch = 0. Time index = 1. Threshold MC = 0.6. Threshold MD = 0.7
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 0. Time index = 1. Threshold MC = 0.6. Threshold MD = 0.8
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 0. Time index = 1. Threshold MC = 0.6. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 0. Time index = 1. Threshold MC = 0.7. Threshold MD = 0.5
RD = 745 NZ = 1316 NZA = 1853 pNZ = 0.038% pNZA = 0.054% AD = 56.61% ADA = 40.21% NO = 519 GSP = 58.19% NSP = 48.94%
SI2 = -25.45 SI = -31

Epoch = 0. Time index = 2. Threshold MC = 0.7. Threshold MD = 0.7
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 0. Time index = 2. Threshold MC = 0.7. Threshold MD = 0.8
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 0. Time index = 2. Threshold MC = 0.7. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 0. Time index = 2. Threshold MC = 0.8. Threshold MD = 0.5
RD = 24 NZ = 32 NZA = 38 pNZ = 0.001% pNZA = 0.001% AD = 75.00% ADA = 63.16% NO = 32 GSP = 71.88% NSP = 68.75%
SI2 = 3.40 SI = 4.40 eGROI 

...

Epoch = 4. Time index = 2. Threshold MC = 0.7. Threshold MD = 0.7
RD = 184 NZ = 240 NZA = 344 pNZ = 0.007% pNZA = 0.010% AD = 76.67% ADA = 53.49% NO = 159 GSP = 74.21% NSP = 50.31%
SI2 = 18.55 SI = -7.45 eGROI = 5.68% eROI = -2.17% eROI2 = 2.50% eROI3 = 0.91% mSpread = 0.0494%
Epoch = 4. Time index = 2. Threshold MC = 0.7. Threshold MD = 0.8
RD = 88 NZ = 97 NZA = 147 pNZ = 0.003% pNZA = 0.004% AD = 90.72% ADA = 59.86% NO = 82 GSP = 84.15% NSP = 48.78%
SI2 = 13.90 SI = -5.10 eGROI = 4.93% eROI = -0.06% eROI2 = 3.29% eROI3 = 2.47% mSpread = 0.0609%
Epoch = 4. Time index = 2. Threshold MC = 0.7. Threshold MD = 0.9
RD = 3 NZ = 3 NZA = 9 pNZ = 0.000% pNZA = 0.000% AD = 100.00% ADA = 33.33% NO = 6 GSP = 83.33% NSP = 33.33%
SI2 = -0.30 SI = -1.30 eGROI = 0.38% eROI = 0.16% eROI2 = 0.26% eROI3 = 0.20% mSpread = 0.0358%

Epoch = 4. Time index = 2. Threshold MC = 0.8. Threshold MD = 0.5
RD = 81 NZ = 124 NZA = 160 pNZ = 0.004% pNZA = 0.005% AD = 65.32% ADA = 50.62% NO = 110 GSP = 59.09% NSP = 45.

Epoch = 4. Time index = 3. Threshold MC = 0.8. Threshold MD = 0.7
RD = 34 NZ = 39 NZA = 52 pNZ = 0.001% pNZA = 0.002% AD = 87.18% ADA = 65.38% NO = 38 GSP = 78.95% NSP = 44.74%
SI2 = 6.10 SI = -3.90 eGROI = 2.22% eROI = -0.09% eROI2 = 1.46% eROI3 = 1.08% mSpread = 0.0606%
Epoch = 4. Time index = 3. Threshold MC = 0.8. Threshold MD = 0.8
RD = 24 NZ = 26 NZA = 35 pNZ = 0.001% pNZA = 0.001% AD = 92.31% ADA = 68.57% NO = 29 GSP = 86.21% NSP = 48.28%
SI2 = 6.05 SI = -1.95 eGROI = 2.45% eROI = 0.57% eROI2 = 1.87% eROI3 = 1.58% mSpread = 0.0646%
Epoch = 4. Time index = 3. Threshold MC = 0.8. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 2 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 100.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.02% eROI = -0.01% eROI2 = -0.00% eROI3 = -0.01% mSpread = 0.0245%

Epoch = 4. Time index = 3. Threshold MC = 0.9. Threshold MD = 0.5
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI 

Epoch = 5. Time index = 0. Threshold MC = 0.8. Threshold MD = 0.7
RD = 21 NZ = 29 NZA = 37 pNZ = 0.001% pNZA = 0.001% AD = 72.41% ADA = 56.76% NO = 24 GSP = 70.83% NSP = 54.17%
SI2 = 0.80 SI = -0.20 eGROI = 0.91% eROI = 0.12% eROI2 = 0.43% eROI3 = 0.19% mSpread = 0.0329%
Epoch = 5. Time index = 0. Threshold MC = 0.8. Threshold MD = 0.8
RD = 8 NZ = 11 NZA = 12 pNZ = 0.000% pNZA = 0.000% AD = 72.73% ADA = 66.67% NO = 11 GSP = 72.73% NSP = 63.64%
SI2 = 1.95 SI = 0.95 eGROI = 1.00% eROI = 0.53% eROI2 = 0.78% eROI3 = 0.67% mSpread = 0.0434%
Epoch = 5. Time index = 0. Threshold MC = 0.8. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 5. Time index = 0. Threshold MC = 0.9. Threshold MD = 0.5
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 

Epoch = 5. Time index = 1. Threshold MC = 0.9. Threshold MD = 0.7
RD = 0 NZ = 0 NZA = 1 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = -0.01% eROI = -0.08% eROI2 = -0.03% eROI3 = -0.04% mSpread = 0.0700%
Epoch = 5. Time index = 1. Threshold MC = 0.9. Threshold MD = 0.8
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 5. Time index = 1. Threshold MC = 0.9. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 5. Time index = 2. Threshold MC = 0.5. Threshold MD = 0.5
J_test = 4.2360787, J_train = 4.2787776463662235, Accuracy=0.7322990115514276
RD = 12482 NZ = 22397 NZA = 39103 pNZ = 0.648% pNZA = 1.

Epoch = 5. Time index = 3. Threshold MC = 0.5. Threshold MD = 0.7
RD = 1241 NZ = 1689 NZA = 3250 pNZ = 0.049% pNZA = 0.094% AD = 73.48% ADA = 38.18% NO = 902 GSP = 73.39% NSP = 41.69%
SI2 = 38.90 SI = -120.10 eGROI = 27.82% eROI = -13.54% eROI2 = 9.78% eROI3 = 0.76% mSpread = 0.0458%
Epoch = 5. Time index = 3. Threshold MC = 0.5. Threshold MD = 0.8
RD = 441 NZ = 551 NZA = 971 pNZ = 0.016% pNZA = 0.028% AD = 80.04% ADA = 45.42% NO = 325 GSP = 77.23% NSP = 43.38%
SI2 = 31.25 SI = -37.75 eGROI = 10.15% eROI = -7.21% eROI2 = 3.65% eROI3 = 0.40% mSpread = 0.0534%
Epoch = 5. Time index = 3. Threshold MC = 0.5. Threshold MD = 0.9
RD = 8 NZ = 8 NZA = 26 pNZ = 0.000% pNZA = 0.001% AD = 100.00% ADA = 30.77% NO = 13 GSP = 76.92% NSP = 30.77%
SI2 = -0.15 SI = -3.15 eGROI = 0.41% eROI = -0.64% eROI2 = 0.15% eROI3 = 0.02% mSpread = 0.0807%

Epoch = 5. Time index = 3. Threshold MC = 0.6. Threshold MD = 0.5
RD = 3490 NZ = 6206 NZA = 9479 pNZ = 0.180% pNZA = 0.274% AD = 56.24% ADA = 36.82% NO = 1480 GS

In [1]:
# Function Build IO from var
def build_IO_from_var_notebook(data, model, stats, IO, totalSampsPerLevel, features, returns, symbols, calculate_roi):
    # total number of possible channels
    nExS = data.nEventsPerStat
    mW = data.movingWindow
    nChannels = int(nExS/mW)
    # sequence length
    seq_len = model.seq_len#int((data.lB-data.nEventsPerStat)/data.movingWindow)
    # samples allocation per batch
    aloc = 2**20
    # extract means and stats
    means_in = stats['means_in']
    stds_in = stats['stds_in']
    #m_in = stats['m_in']
    stds_out = stats['stds_out']
    #m_out = stats['m_out']
    #print("m_in")
    #print(m_in)
    #print("m_out")
    #print(m_out)
    # add dateTimes, bids and asks if are included in file
    if calculate_roi:
        dts = symbols['DT']
        bids = symbols['B']
        asks = symbols['A']

        D = IO['D']
        B = IO['B']
        A = IO['A']

    # extract IO structures
    X = IO['X']
    Y = IO['Y']
    #I = IO['I']
    pointer = IO['pointer']

    feats_var_normed = np.minimum(np.maximum((features-means_in)/\
                         stds_in,-data.max_var),data.max_var)
    # get some scalars
    nSamps = feats_var_normed.shape[0]
    print("features.shape")
    print(features.shape)
    print("nSamps")
    print(nSamps)
    samp_remaining = nSamps-2*nChannels-seq_len+1
    print("samp_remaining")
    print(samp_remaining)
    print("returns.shape")
    print(returns.shape)
    chunks = int(np.ceil(samp_remaining/aloc))
    # init counter of samps processed
    offset = 0
    # loop over chunks
    for i in range(chunks):
        # this batch length
        batch = np.min([samp_remaining, aloc])
        # create support numpy vectors to speed up iterations
        v_support = feats_var_normed[offset:offset+batch+seq_len, :]
        # get init and end indexes for returns
        init_idx_rets = nChannels+offset+seq_len-1
        print("init_idx_rets")
        print(init_idx_rets)
        end_idx_rets = nChannels+offset+batch+seq_len-1#+2*seq_len-1
        print("end_idx_rets")
        print(end_idx_rets)
        r_support = returns[init_idx_rets:end_idx_rets, data.lookAheadIndex]
        print("r_support.shape")
        print(r_support.shape)
        assert(r_support.shape[0]==end_idx_rets-init_idx_rets)
        # we only take here the entry time index, and later at DTA building the 
        # exit time index is derived from the entry time and the number of events to
        # look ahead
        if calculate_roi:
            dt_support = dts[init_idx_rets:end_idx_rets, [0,data.lookAheadIndex+1]]
            b_support = bids[init_idx_rets:end_idx_rets, [0,data.lookAheadIndex+1]]
            a_support = asks[init_idx_rets:end_idx_rets, [0,data.lookAheadIndex+1]]
        # update remaining samps to proceed
        samp_remaining = samp_remaining-batch
        # init formatted input and output
        X_i = np.zeros((batch, seq_len, features.shape[1]))
        # real-valued output
        O_i = np.zeros((batch, seq_len, 1))    
        if calculate_roi:
            # last dimension is to incorporate in and out symbols
            D_i = np.chararray((batch, 2),itemsize=19)
            B_i = np.zeros((batch, 2))
            A_i = np.zeros((batch, 2))

        for nI in range(batch):
            # get input
            v_s_s = v_support[nI:nI+seq_len, :]
            X_i[nI,:,:] = v_s_s[::-1,:]#v_support[nI:nI+seq_len, :]            
            # due to substraction of features for variation, output gets the 
            # feature one entry later
            O_i[nI,:,0] = r_support[nI]
            if calculate_roi:
                D_i[nI,:] = dt_support[nI,:]
                B_i[nI,:] = b_support[nI,:]
                A_i[nI,:] = a_support[nI,:]

        # normalize output
        O_i = O_i/stds_out[data.lookAheadIndex]
        # update counters
        offset = offset+batch
        # get decimal and binary outputs
        Y_i, y_dec = _build_bin_output(model, O_i, batch)
        # get samples per level
        for l in range(model.size_output_layer):
            totalSampsPerLevel[l] = totalSampsPerLevel[l]+np.sum(y_dec[:,-1,0]==l)
        # resize IO structures
        X.resize((pointer+batch, seq_len,features.shape[1]))
        Y.resize((pointer+batch, seq_len,model.commonY+model.size_output_layer))
        # update IO structures
        X[pointer:pointer+batch,:,:] = X_i
        Y[pointer:pointer+batch,:,:] = Y_i
        if calculate_roi:
            # resize
            D.resize((pointer+batch, 2))
            B.resize((pointer+batch, 2))
            A.resize((pointer+batch, 2))
            # update
            D[pointer:pointer+batch,:] = D_i
            B[pointer:pointer+batch,:] = B_i
            A[pointer:pointer+batch,:] = A_i
    #        save_as_matfile('X_h_n_'+str(int(s/2)),'X_h_n'+str(int(s/2)),X_i)
    #        save_as_matfile('O_h_n_'+str(int(s/2)),'O_h_n'+str(int(s/2)),O_i)

        # uodate pointer
        pointer += batch
    # end of for i in range(chunks):
    # update dictionary
    IO['X'] = X
    IO['Y'] = Y
    IO['pointer'] = pointer
    if calculate_roi:
        IO['D'] = D
        IO['B'] = B
        IO['A'] = A
    
    return IO, totalSampsPerLevel
    

In [4]:
# Snippet Build IO from var

# total number of possible channels
nChannels = int(nExS/mW)
# sequence length
seq_len = model.seq_len#int((data.lB-data.nEventsPerStat)/data.movingWindow)
# samples allocation per batch
aloc = 2**20
# extract means and stats
means_in = stats['means_in']
stds_in = stats['stds_in']
m_in = stats['m_in']
stds_out = stats['stds_out']
m_out = stats['m_out']
#print("m_in")
#print(m_in)
#print("m_out")
#print(m_out)
# add dateTimes, bids and asks if are included in file
all_info = 0
if calculate_roi:
    all_info = 1
    dts = symbols['DT']
    bids = symbols['B']
    asks = symbols['A']
        
    D = IO['D']
    B = IO['B']
    A = IO['A']

# extract IO structures
X = IO['X']
Y = IO['Y']
#I = IO['I']
pointer = IO['pointer']

feats_var_normed = np.minimum(np.maximum((features-means_in)/\
                     stds_in,-data.max_var),data.max_var)
# get some scalars
nSamps = feats_var_normed.shape[0]
samp_remaining = nSamps-nChannels-seq_len-1
print("samp_remaining")
print(samp_remaining)
chunks = int(np.ceil(samp_remaining/aloc))
# init counter of samps processed
offset = 0
# loop over chunks
for i in range(chunks):
    # this batch length
    batch = np.min([samp_remaining,aloc])
    print("batch")
    print(batch)
    # create support numpy vectors to speed up iterations
    v_support = feats_var_normed[offset:offset+batch+seq_len, :]
    # get init and end indexes for returns
    init_idx_rets = nChannels+offset+seq_len-1
    end_idx_rets = nChannels+offset+batch+2*seq_len-1
    r_support = returns[init_idx_rets:end_idx_rets, data.lookAheadIndex]
    print("returns.shape")
    print(returns.shape)
    print("nChannels+offset+2")
    print(nChannels+offset+2)
    print("nChannels+offset+batch+seq_len+2")
    print(nChannels+offset+batch+seq_len+2)
    print("r_support.shape")
    print(r_support.shape)
    print("nChannels+offset+batch+seq_len+2-(nChannels+offset+2)")
    print(nChannels+offset+batch+seq_len+2-(nChannels+offset+2))
    print("returns[-1,data.lookAheadIndex]")
    print(returns[-1,data.lookAheadIndex])
    print("r_support[-1]")
    print(r_support[-1])
    # we only take here the entry time index, and later at DTA building the 
    # exit time index is derived from the entry time and the number of events to
    # look ahead
    if calculate_roi:
        dt_support = dts[init_idx_rets:end_idx_rets, [0,data.lookAheadIndex+1]]
        b_support = bids[init_idx_rets:end_idx_rets, [0,data.lookAheadIndex+1]]
        a_support = asks[init_idx_rets:end_idx_rets, [0,data.lookAheadIndex+1]]
    # update remaining samps to proceed
    samp_remaining = samp_remaining-batch
    # init formatted input and output
    X_i = np.zeros((batch, seq_len, features.shape[1]))
    # real-valued output
    O_i = np.zeros((batch, seq_len, 1))    
    if calculate_roi:
        # last dimension is to incorporate in and out symbols
        D_i = np.chararray((batch, 2),itemsize=19)
        B_i = np.zeros((batch, 2))
        A_i = np.zeros((batch, 2))
    
    for nI in range(batch):
        # get input
        v_s_s = v_support[nI:nI+seq_len, :]
        X_i[nI,:,:] = v_s_s[::-1,:]#v_support[nI:nI+seq_len, :]            
        # due to substraction of features for variation, output gets the 
        # feature one entry later
        O_i[nI,:,0] = r_support[nI]
        if calculate_roi:
            D_i[nI,:] = dt_support[nI,:]
            B_i[nI,:] = b_support[nI,:]
            A_i[nI,:] = a_support[nI,:]
    
    # normalize output
    print("stds_out.shape")
    print(stds_out.shape)
    O_i = O_i/stds_out[data.lookAheadIndex]
    # update counters
    offset = offset+batch
    # get decimal and binary outputs
    Y_i, y_dec = _build_bin_output(model, O_i, batch)
    # get samples per level
    for l in range(model.size_output_layer):
        totalSampsPerLevel[l] = totalSampsPerLevel[l]+np.sum(y_dec[:,-1,0]==l)
    # resize IO structures
    X.resize((pointer+batch, seq_len,features.shape[1]))
    Y.resize((pointer+batch, seq_len,model.commonY+model.size_output_layer))
    # update IO structures
    X[pointer:pointer+batch,:,:] = X_i
    Y[pointer:pointer+batch,:,:] = Y_i
    if calculate_roi:
        # resize
        D.resize((pointer+batch, 2))
        B.resize((pointer+batch, 2))
        A.resize((pointer+batch, 2))
        # update
        D[pointer:pointer+batch,:] = D_i
        B[pointer:pointer+batch,:] = B_i
        A[pointer:pointer+batch,:] = A_i
#        save_as_matfile('X_h_n_'+str(int(s/2)),'X_h_n'+str(int(s/2)),X_i)
#        save_as_matfile('O_h_n_'+str(int(s/2)),'O_h_n'+str(int(s/2)),O_i)
        
    # uodate pointer
    pointer += batch
# end of for i in range(chunks):
# update dictionary
IO['X'] = X
IO['Y'] = Y
IO['pointer'] = pointer
if calculate_roi:
    IO['D'] = D
    IO['B'] = B
    IO['A'] = A
print("totalSampsPerLevel")
print(totalSampsPerLevel)

if if_build_IO:
    ass_IO_ass[ass_idx] = IO['pointer']
    #print("\tTime for "+thisAsset+":"+str(np.floor(time.time()-tic))+"s"+
    #          ". Total time:"+str(np.floor(time.time()-ticTotal))+"s")
# end of assets loop?
if if_build_IO:
    f_IO.attrs.create('ass_IO_ass', ass_IO_ass, dtype=int)
    f_IO.close()
else:
    # get ass_IO_ass from disk
    f_IO = h5py.File(filename_IO,'r')
    ass_IO_ass = f_IO.attrs.get("ass_IO_ass")
    f_IO.close()
m_t = IO['pointer']
print("m_t")
print(m_t)
print("IO['pointer']")
print(IO['pointer'])

m_in
42343
m_out
42333
samp_remaining
42319
batch
42319
returns.shape
(42333, 4)
nChannels+offset+2
12
nChannels+offset+batch+seq_len+2
42334
r_support.shape
(42321,)
nChannels+offset+batch+seq_len+2-(nChannels+offset+2)
42322
returns[-1,data.lookAheadIndex]
9.999999999998899e-05
r_support[-1]
9.999999999998899e-05
stds_out.shape
(4,)
totalSampsPerLevel
[  320.  6215. 29138.  6239.   407.]
m_t
42319
IO['pointer']
42319


In [5]:
f_IO.close()

In [6]:
O_i.shape

(42319, 3, 1)

In [20]:
# Train
import tensorflow as tf
from inputs import build_IO_from_var_wrapper
from config import retrieve_config
import numpy as np
def build_and_train(*ins):
    """  """
    if len(ins)>0:
        config = ins[0]
    else:
        config = retrieve_config('C0400')
    
    if 'weights_directory' in config:
        weights_directory = config['weights_directory']
    else:
        weights_directory = '../RNN/weights/'
    
    model, m_t, filename_IO, _ = build_IO_from_var_wrapper('tr', config)
    # reset graph
    alloc = 2**20
    tf.reset_default_graph()
    # start session
    with tf.Session() as sess:    
        model.train(sess, int(np.ceil(m_t/alloc)), weights_directory,
                    ID=config['IDweights'], IDIO=config['IDweights'], 
                    data_format='hdf5', filename_IO=filename_IO, aloc=alloc)

In [19]:
# Test
import tensorflow as tf
from inputs import build_IO_from_var_wrapper
from config import retrieve_config
%load_ext autoreload
%autoreload 2

def build_and_test(*ins):
    """  """
    if len(ins)>0:
        config = ins[0]
    else:
        config = retrieve_config('C0400')
        
    if 'weights_directory' in config:
        weights_directory = config['weights_directory']
    else:
        weights_directory = '../RNN/weights/'
    if 'results_directory' in config:
        results_directory = config['results_directory']
    else:
        results_directory = '../RNN/results/'
        
    model, m_t, filename_IO, DTA = build_IO_from_var_wrapper('te', config)
    
    alloc = 200000
    # reset graph
    tf.reset_default_graph()
    # start session
    with tf.Session() as sess:
        # run test RNN
        print("IDresults: "+config['IDresults'])
        
        model.test2(sess, config['dateTest'], config['IDresults'], config['IDweights'], 
                    alloc, 'test', weights_directory, filename_IO, results_directory,
                    startFrom=config['startFrom'], data_format='hdf5', DTA=DTA, 
                    save_journal=config['save_journal'], endAt=config['endAt'], from_var=True)
    print("DONE")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# run train/test alternatively
def run_train_test(config, its, if_train, if_test, if_get_features):
    """
    
    """
    if if_get_features:
        get_features(config)
    # loop over iteratuibs
    for it in range(its):
        print("Iteration {0:d} of {1:d}".format(it,its-1))
        # control build_IO
        if it>0 and config['if_build_IO']:
            config['if_build_IO'] = False
        if if_train:
            print("IDweights: "+config['IDweights'])
        if if_test:
            print("IDresults: "+config['IDresults'])
        # launch train
        if if_train:
            # here we should check if the HDF5 file is used
            build_and_train(config)
        # launch test
        if if_test:
            build_and_test(config)

In [4]:
from config import configuration
# define config structure
entries = {'config_name':'C0400',
          
          'movingWindow':100,
          'nEventsPerStat':1000,
          'lB':1300,
          'feature_keys_manual':[],
          'feature_keys_tsfresh':[],
           'var_feat_keys':[i for i in range(68,93)],
          
          'if_build_IO':False,
          'from_stats_file':False,
          'IDweights':'000400',
          'inverse_load':True,
          
          'IDresults':'100400',
          'startFrom':-1,
          'endAt':-1,
          'save_journal':False}

config = configuration(entries)

WARNING! Arguments not taken into consideration
Config file ../config/C0400.config exists. Loaded from disk


In [ ]:
# Automate

# init config
if_get_features = False
if_train = True
if_test = True
its = 20
configs = [config['config_name']]

configs_list = []
# load configuration files
for config_name in configs:
    configs_list.append(retrieve_config(config_name))
# run train/test
for config in configs_list:
    # set automation-specific config fields
    config['num_epochs'] = 1
    config['startFrom'] = -1
    config['endAt'] = -1
    run_train_test(config, its, if_train, if_test, if_get_features)

Config file ../config/C0400.config loaded from disk
Iteration 0 of 19
IDweights: 000400
IDresults: 100400
1 AUDCAD
2 EURAUD
3 EURCAD
4 EURCHF
7 EURGBP
8 EURNZD
10 EURUSD
11 GBPAUD
12 GBPCAD
13 GBPCHF
14 GBPUSD
15 GOLD
16 USDCAD
17 USDCHF
19 USDJPY
27 CADJPY
28 EURJPY
29 AUDJPY
30 CHFJPY
31 GBPJPY
32 NZDUSD
ID = 000400
INFO:tensorflow:Restoring parameters from ../RNN/weights/000400/000008/000008
Parameters loaded. Epoch 8
Training from epoch 9 till 9
Time training Chunck 0 of 14: 400 s
Time training Chunck 1 of 14: 406 s
Time training Chunck 2 of 14: 408 s
Time training Chunck 3 of 14: 411 s
Time training Chunck 4 of 14: 413 s
Time training Chunck 5 of 14: 411 s
Time training Chunck 6 of 14: 410 s
Time training Chunck 7 of 14: 417 s
Time training Chunck 8 of 14: 431 s
Time training Chunck 9 of 14: 445 s
Time training Chunck 10 of 14: 410 s
Time training Chunck 11 of 14: 414 s
Time training Chunck 12 of 14: 410 s
Time training Chunck 13 of 14: 416 s
Time training Chunck 14 of 14: 373 s
C

C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\fromnumeric.py:3194: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch = 9. Time index = 0. Threshold MC = 0.8. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 9. Time index = 0. Threshold MC = 0.9. Threshold MD = 0.5
RD = 6 NZ = 9 NZA = 13 pNZ = 0.000% pNZA = 0.000% AD = 66.67% ADA = 46.15% NO = 13 GSP = 61.54% NSP = 53.85%
SI2 = -1.15 SI = -0.15 eGROI = -0.03% eROI = -0.52% eROI2 = -0.29% eROI3 = -0.42% mSpread = 0.0375%
Epoch = 9. Time index = 0. Threshold MC = 0.9. Threshold MD = 0.6
RD = 2 NZ = 2 NZA = 4 pNZ = 0.000% pNZA = 0.000% AD = 100.00% ADA = 50.00% NO = 4 GSP = 50.00% NSP = 50.00%
SI2 = -0.20 SI = -0.20 eGROI = 0.32% eROI = 0.06% eROI2 = 0.24% eROI3 = 0.20% mSpread = 0.0659%
Epoch = 9. Time index = 0. Threshold MC = 0.9. Threshold MD = 0.7
RD = 2 NZ = 2 NZA = 3 pNZ = 0.000% pNZA = 0.000% AD = 100.00% ADA = 66.67% NO = 3 GSP = 66.67% NSP = 66.67%
SI2 = -0.65 SI 

Epoch = 9. Time index = 1. Threshold MC = 0.9. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 9. Time index = 2. Threshold MC = 0.5. Threshold MD = 0.5
J_test = 4.230769, J_train = 4.275993016043996, Accuracy=0.7322263974525469
RD = 12084 NZ = 21689 NZA = 37938 pNZ = 0.627% pNZA = 1.098% AD = 55.71% ADA = 31.85% NO = 4176 GSP = 59.43% NSP = 44.97%
SI2 = -266.80 SI = -418.80 eGROI = 55.64% eROI = -54.80% eROI2 = -27.88% eROI3 = -69.64% mSpread = 0.0264%
Epoch = 9. Time index = 2. Threshold MC = 0.5. Threshold MD = 0.6
RD = 4033 NZ = 6462 NZA = 11238 pNZ = 0.187% pNZA = 0.325% AD = 62.41% ADA = 35.89% NO = 2149 GSP = 64.91% NSP = 45.74%
SI2 = -28.95 SI = -198.95 eGROI = 45.57% eROI = -21.32% eROI2 = 2.59% eROI3 = -18.90% mSpread = 0.0311%
Epoch = 9. Time index = 2. Threshold MC = 0.5. Threshold MD = 0.7
RD = 15

Epoch = 9. Time index = 3. Threshold MC = 0.5. Threshold MD = 0.8
RD = 619 NZ = 798 NZA = 1436 pNZ = 0.023% pNZA = 0.042% AD = 77.57% ADA = 43.11% NO = 466 GSP = 77.90% NSP = 49.79%
SI2 = 56.70 SI = -24.30 eGROI = 20.22% eROI = -2.44% eROI2 = 10.90% eROI3 = 6.24% mSpread = 0.0486%
Epoch = 9. Time index = 3. Threshold MC = 0.5. Threshold MD = 0.9
RD = 64 NZ = 76 NZA = 113 pNZ = 0.002% pNZA = 0.003% AD = 84.21% ADA = 56.64% NO = 36 GSP = 86.11% NSP = 66.67%
SI2 = 7.20 SI = 4.20 eGROI = 1.91% eROI = 0.07% eROI2 = 1.19% eROI3 = 0.83% mSpread = 0.0511%

Epoch = 9. Time index = 3. Threshold MC = 0.6. Threshold MD = 0.5
RD = 3495 NZ = 6389 NZA = 9920 pNZ = 0.185% pNZA = 0.287% AD = 54.70% ADA = 35.23% NO = 1712 GSP = 59.70% NSP = 48.25%
SI2 = -65.60 SI = -115.60 eGROI = 24.97% eROI = -20.75% eROI2 = -9.27% eROI3 = -26.39% mSpread = 0.0267%
Epoch = 9. Time index = 3. Threshold MC = 0.6. Threshold MD = 0.6
RD = 1432 NZ = 2348 NZA = 3747 pNZ = 0.068% pNZA = 0.108% AD = 60.99% ADA = 38.22% NO = 8

...

Epoch = 15. Time index = 3. Threshold MC = 0.7. Threshold MD = 0.5
RD = 962 NZ = 1617 NZA = 2396 pNZ = 0.047% pNZA = 0.069% AD = 59.49% ADA = 40.15% NO = 773 GSP = 57.96% NSP = 48.12%
SI2 = -26.15 SI = -53.15 eGROI = 13.09% eROI = -8.59% eROI2 = -2.37% eROI3 = -10.10% mSpread = 0.0281%
Epoch = 15. Time index = 3. Threshold MC = 0.7. Threshold MD = 0.6
RD = 546 NZ = 866 NZA = 1316 pNZ = 0.025% pNZA = 0.038% AD = 63.05% ADA = 41.49% NO = 565 GSP = 59.12% NSP = 47.96%
SI2 = -10.75 SI = -39.75 eGROI = 12.32% eROI = -4.63% eROI2 = 1.02% eROI3 = -4.63% mSpread = 0.0300%
Epoch = 15. Time index = 3. Threshold MC = 0.7. Threshold MD = 0.7
RD = 274 NZ = 418 NZA = 658 pNZ = 0.012% pNZA = 0.019% AD = 65.55% ADA = 41.64% NO = 279 GSP = 62.37% NSP = 49.10%
SI2 = 0.55 SI = -16.45 eGROI = 8.97% eROI = -0.82% eROI2 = 3.39% eROI3 = 0.60% mSpread = 0.0351%
Epoch = 15. Time index = 3. Threshold MC = 0.7. Threshold MD = 0.8
RD = 159 NZ = 216 NZA = 354 pNZ = 0.006% pNZA = 0.010% AD = 73.61% ADA = 44.92% NO 

Epoch = 16. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.7
RD = 1898 NZ = 2778 NZA = 4945 pNZ = 0.080% pNZA = 0.143% AD = 68.32% ADA = 38.38% NO = 1230 GSP = 67.72% NSP = 44.72%
SI2 = 16.50 SI = -126.50 eGROI = 29.47% eROI = -17.81% eROI2 = 4.87% eROI3 = -7.43% mSpread = 0.0384%
Epoch = 16. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.8
RD = 720 NZ = 964 NZA = 1693 pNZ = 0.028% pNZA = 0.049% AD = 74.69% ADA = 42.53% NO = 544 GSP = 72.24% NSP = 47.79%
SI2 = 35.80 SI = -39.20 eGROI = 16.71% eROI = -7.38% eROI2 = 5.83% eROI3 = 0.39% mSpread = 0.0443%
Epoch = 16. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.9
RD = 91 NZ = 99 NZA = 189 pNZ = 0.003% pNZA = 0.005% AD = 91.92% ADA = 48.15% NO = 70 GSP = 91.43% NSP = 45.71%
SI2 = 15.50 SI = -6.50 eGROI = 3.78% eROI = -0.26% eROI2 = 2.38% eROI3 = 1.68% mSpread = 0.0577%

Epoch = 16. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.5
RD = 3228 NZ = 5552 NZA = 8578 pNZ = 0.161% pNZA = 0.248% AD = 58.14% ADA = 37.63% NO 

Epoch = 16. Time index = 1. Threshold MC = 0.6. Threshold MD = 0.6
RD = 1450 NZ = 2343 NZA = 3752 pNZ = 0.068% pNZA = 0.109% AD = 61.89% ADA = 38.65% NO = 1068 GSP = 62.36% NSP = 46.91%
SI2 = -6.40 SI = -86.40 eGROI = 22.67% eROI = -11.46% eROI2 = 1.31% eROI3 = -9.37% mSpread = 0.0320%
Epoch = 16. Time index = 1. Threshold MC = 0.6. Threshold MD = 0.7
RD = 825 NZ = 1237 NZA = 1982 pNZ = 0.036% pNZA = 0.057% AD = 66.69% ADA = 41.62% NO = 633 GSP = 67.14% NSP = 49.45%
SI2 = 19.85 SI = -35.15 eGROI = 18.19% eROI = -4.59% eROI2 = 5.53% eROI3 = -0.80% mSpread = 0.0360%
Epoch = 16. Time index = 1. Threshold MC = 0.6. Threshold MD = 0.8
RD = 357 NZ = 495 NZA = 780 pNZ = 0.014% pNZA = 0.023% AD = 72.12% ADA = 45.77% NO = 303 GSP = 72.61% NSP = 49.83%
SI2 = 17.35 SI = -15.65 eGROI = 10.91% eROI = -1.52% eROI2 = 4.85% eROI3 = 1.82% mSpread = 0.0410%
Epoch = 16. Time index = 1. Threshold MC = 0.6. Threshold MD = 0.9
RD = 69 NZ = 99 NZA = 187 pNZ = 0.003% pNZA = 0.005% AD = 69.70% ADA = 36.90% NO 

Epoch = 16. Time index = 2. Threshold MC = 0.7. Threshold MD = 0.5
RD = 829 NZ = 1380 NZA = 2015 pNZ = 0.040% pNZA = 0.058% AD = 60.07% ADA = 41.14% NO = 732 GSP = 58.06% NSP = 47.27%
SI2 = -26.60 SI = -56.60 eGROI = 9.80% eROI = -10.77% eROI2 = -4.84% eROI3 = -12.16% mSpread = 0.0281%
Epoch = 16. Time index = 2. Threshold MC = 0.7. Threshold MD = 0.6
RD = 461 NZ = 724 NZA = 1115 pNZ = 0.021% pNZA = 0.032% AD = 63.67% ADA = 41.35% NO = 485 GSP = 58.56% NSP = 47.42%
SI2 = -16.75 SI = -36.75 eGROI = 10.49% eROI = -4.23% eROI2 = 0.79% eROI3 = -4.06% mSpread = 0.0303%
Epoch = 16. Time index = 2. Threshold MC = 0.7. Threshold MD = 0.7
RD = 271 NZ = 402 NZA = 629 pNZ = 0.012% pNZA = 0.018% AD = 67.41% ADA = 43.08% NO = 286 GSP = 62.59% NSP = 47.90%
SI2 = -2.30 SI = -20.30 eGROI = 8.07% eROI = -1.88% eROI2 = 2.35% eROI3 = -0.51% mSpread = 0.0348%
Epoch = 16. Time index = 2. Threshold MC = 0.7. Threshold MD = 0.8
RD = 156 NZ = 200 NZA = 326 pNZ = 0.006% pNZA = 0.009% AD = 78.00% ADA = 47.85% N

Epoch = 16. Time index = 3. Threshold MC = 0.7. Threshold MD = 0.9
RD = 62 NZ = 79 NZA = 153 pNZ = 0.002% pNZA = 0.004% AD = 78.48% ADA = 40.52% NO = 57 GSP = 80.70% NSP = 59.65%
SI2 = 9.65 SI = 2.65 eGROI = 3.07% eROI = 0.42% eROI2 = 1.93% eROI3 = 1.36% mSpread = 0.0466%

Epoch = 16. Time index = 3. Threshold MC = 0.8. Threshold MD = 0.5
RD = 77 NZ = 125 NZA = 196 pNZ = 0.004% pNZA = 0.006% AD = 61.60% ADA = 39.29% NO = 130 GSP = 51.54% NSP = 40.77%
SI2 = -13.50 SI = -18.50 eGROI = 0.22% eROI = -3.51% eROI2 = -2.38% eROI3 = -3.68% mSpread = 0.0287%
Epoch = 16. Time index = 3. Threshold MC = 0.8. Threshold MD = 0.6
RD = 53 NZ = 82 NZA = 135 pNZ = 0.002% pNZA = 0.004% AD = 64.63% ADA = 39.26% NO = 91 GSP = 51.65% NSP = 43.96%
SI2 = -6.05 SI = -10.05 eGROI = 0.58% eROI = -2.31% eROI2 = -1.24% eROI3 = -2.15% mSpread = 0.0318%
Epoch = 16. Time index = 3. Threshold MC = 0.8. Threshold MD = 0.7
RD = 30 NZ = 47 NZA = 83 pNZ = 0.001% pNZA = 0.002% AD = 63.83% ADA = 36.14% NO = 56 GSP = 51.79% 

In [21]:
build_and_test(config)

1 AUDCAD
2 EURAUD
3 EURCAD
4 EURCHF
7 EURGBP
8 EURNZD
10 EURUSD
11 GBPAUD
12 GBPCAD
13 GBPCHF
14 GBPUSD
15 GOLD
16 USDCAD
17 USDCHF
19 USDJPY
27 CADJPY
28 EURJPY
29 AUDJPY
30 CHFJPY
31 GBPJPY
32 NZDUSD
IDresults: 100400
INFO:tensorflow:Restoring parameters from ../RNN/weights/000400/000008/000008
Parameters loaded. Epoch 8
Epoch 8 of 8. Getting output...
Chunck 1 of 18
Chunck 2 of 18
Chunck 3 of 18
Chunck 4 of 18
Chunck 5 of 18
Chunck 6 of 18
Chunck 7 of 18
Chunck 8 of 18
Chunck 9 of 18
Chunck 10 of 18
Chunck 11 of 18
Chunck 12 of 18
Chunck 13 of 18
Chunck 14 of 18
Chunck 15 of 18
Chunck 16 of 18
Chunck 17 of 18
Chunck 18 of 18
Getting results
Epoch = 8. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.5
J_test = 4.231582, J_train = 4.276547891847993, Accuracy=0.7322995901498252
RD = 13551 NZ = 24269 NZA = 42845 pNZ = 0.702% pNZA = 1.240% AD = 55.84% ADA = 31.63% NO = 4359 GSP = 59.74% NSP = 44.80%
SI2 = -281.45 SI = -444.45 eGROI = 47.62% eROI = -69.07% eROI2 = -39.56% eROI3 = -83

C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\fromnumeric.py:3194: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch = 8. Time index = 0. Threshold MC = 0.9. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 8. Time index = 1. Threshold MC = 0.5. Threshold MD = 0.5
J_test = 4.231582, J_train = 4.276547891847993, Accuracy=0.7322634277499842
RD = 11585 NZ = 20643 NZA = 35718 pNZ = 0.597% pNZA = 1.033% AD = 56.12% ADA = 32.43% NO = 3926 GSP = 60.47% NSP = 46.46%
SI2 = -211.30 SI = -335.30 eGROI = 59.78% eROI = -42.50% eROI2 = -18.74% eROI3 = -58.00% mSpread = 0.0261%
Epoch = 8. Time index = 1. Threshold MC = 0.5. Threshold MD = 0.6
RD = 3796 NZ = 6016 NZA = 10387 pNZ = 0.174% pNZA = 0.300% AD = 63.10% ADA = 36.55% NO = 2079 GSP = 63.68% NSP = 46.37%
SI2 = -33.45 SI = -179.45 eGROI = 43.99% eROI = -19.53% eROI2 = 2.41% eROI3 = -18.38% mSpread = 0.0306%
Epoch = 8. Time index = 1. Threshold MC = 0.5. Threshold MD = 0.7
RD = 14

Epoch = 8. Time index = 2. Threshold MC = 0.5. Threshold MD = 0.8
RD = 536 NZ = 687 NZA = 1186 pNZ = 0.020% pNZA = 0.034% AD = 78.02% ADA = 45.19% NO = 429 GSP = 75.76% NSP = 47.32%
SI2 = 38.05 SI = -32.95 eGROI = 17.21% eROI = -3.82% eROI2 = 8.63% eROI3 = 4.34% mSpread = 0.0490%
Epoch = 8. Time index = 2. Threshold MC = 0.5. Threshold MD = 0.9
RD = 39 NZ = 41 NZA = 58 pNZ = 0.001% pNZA = 0.002% AD = 95.12% ADA = 67.24% NO = 32 GSP = 87.50% NSP = 65.62%
SI2 = 7.40 SI = 3.40 eGROI = 2.40% eROI = 0.67% eROI2 = 1.76% eROI3 = 1.44% mSpread = 0.0540%

Epoch = 8. Time index = 2. Threshold MC = 0.6. Threshold MD = 0.5
RD = 3739 NZ = 6711 NZA = 10353 pNZ = 0.194% pNZA = 0.300% AD = 55.71% ADA = 36.12% NO = 1814 GSP = 59.54% NSP = 48.73%
SI2 = -67.70 SI = -113.70 eGROI = 27.06% eROI = -20.45% eROI2 = -9.22% eROI3 = -27.36% mSpread = 0.0262%
Epoch = 8. Time index = 2. Threshold MC = 0.6. Threshold MD = 0.6
RD = 1468 NZ = 2360 NZA = 3717 pNZ = 0.068% pNZA = 0.108% AD = 62.20% ADA = 39.49% NO = 96

Epoch = 8. Time index = 3. Threshold MC = 0.6. Threshold MD = 0.8
RD = 235 NZ = 313 NZA = 511 pNZ = 0.009% pNZA = 0.015% AD = 75.08% ADA = 45.99% NO = 213 GSP = 77.00% NSP = 51.64%
SI2 = 23.85 SI = -7.15 eGROI = 10.25% eROI = 0.06% eROI2 = 5.99% eROI3 = 3.86% mSpread = 0.0479%
Epoch = 8. Time index = 3. Threshold MC = 0.6. Threshold MD = 0.9
RD = 47 NZ = 51 NZA = 74 pNZ = 0.001% pNZA = 0.002% AD = 92.16% ADA = 63.51% NO = 32 GSP = 78.12% NSP = 59.38%
SI2 = 3.40 SI = 1.40 eGROI = 2.01% eROI = 0.66% eROI2 = 1.37% eROI3 = 1.05% mSpread = 0.0423%

Epoch = 8. Time index = 3. Threshold MC = 0.7. Threshold MD = 0.5
RD = 970 NZ = 1612 NZA = 2313 pNZ = 0.047% pNZA = 0.067% AD = 60.17% ADA = 41.94% NO = 707 GSP = 56.44% NSP = 46.96%
SI2 = -35.85 SI = -56.85 eGROI = 6.29% eROI = -12.72% eROI2 = -7.85% eROI3 = -14.92% mSpread = 0.0269%
Epoch = 8. Time index = 3. Threshold MC = 0.7. Threshold MD = 0.6
RD = 422 NZ = 637 NZA = 962 pNZ = 0.018% pNZA = 0.028% AD = 66.25% ADA = 43.87% NO = 358 GSP = 63.

In [16]:
from config import modify_config

modify_config(config['config_name'],'if_build_IO',False)
config = retrieve_config(config['config_name'])

[autoreload of inputs failed: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\IPython\extensions\autoreload.py", line 384, in superreload
    update_generic(old_obj, new_obj)
  File "C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\IPython\extensions\autoreload.py", line 323, in update_generic
    update(a, b)
  File "C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\IPython\extensions\autoreload.py", line 278, in update_class
    if old_obj == new_obj:
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
]


Config file ../config/C0400.config saved
Config file ../config/C0400.config loaded from disk


In [15]:
build_and_train(config)

1 AUDCAD
2 EURAUD
3 EURCAD
4 EURCHF
7 EURGBP
8 EURNZD
10 EURUSD
11 GBPAUD
12 GBPCAD
13 GBPCHF
14 GBPUSD
15 GOLD
16 USDCAD
17 USDCHF
19 USDJPY
27 CADJPY
28 EURJPY
29 AUDJPY
30 CHFJPY
31 GBPJPY
32 NZDUSD
ID = 000400
INFO:tensorflow:Restoring parameters from ../RNN/weights/000400/000007/000007
Parameters loaded. Epoch 7
Training from epoch 8 till 27
Time training Chunck 0 of 14: 367 s
Time training Chunck 1 of 14: 366 s
Time training Chunck 2 of 14: 375 s
Time training Chunck 3 of 14: 385 s
Time training Chunck 4 of 14: 395 s
Time training Chunck 5 of 14: 387 s
Time training Chunck 6 of 14: 391 s
Time training Chunck 7 of 14: 392 s
Time training Chunck 8 of 14: 388 s
Time training Chunck 9 of 14: 397 s
Time training Chunck 10 of 14: 421 s
Time training Chunck 11 of 14: 398 s
Time training Chunck 12 of 14: 391 s
Time training Chunck 13 of 14: 410 s
Time training Chunck 14 of 14: 360 s
Cost after epoch 8: 0.543258. Av cost 4.276548
17:49:55 Total time training: 97.15m
INFO:tensorflow:../RNN

KeyboardInterrupt: 